In [3]:

# !pip install gensim
# !pip install git+https://github.com/boudinfl/pke.git
# !python -m spacy download en
# !pip install bert-extractive-summarizer --upgrade --force-reinstall
# !pip install spacy==2.1.3 --upgrade --force-reinstall
# !pip install -U nltk
# !pip install -U pywsd

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('popular')

[nltk_data] Downloading package stopwords to /Users/anhong-
[nltk_data]     eun/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /Users/anhong-
[nltk_data]    |     eun/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /Users/anhong-
[nltk_data]    |     eun/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /Users/anhong-
[nltk_data]    |     eun/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /Users/anhong-
[nltk_data]    |     eun/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /Users/anhong-
[nltk_data]    |     eun/nltk_data...
[nltk_data]    |   Package inaugural is 

True

## Pytorch
https://pytorch.org/get-started/locally/<br>
여기서 각자 pc환경에 맞게 다운로드하기
꼭 환경에 맞아야 함 !!!!!!
아직 gpu사용안하니까 쿠다는 안써용

In [5]:
# conda install pytorch torchvision torchaudio -c pytorch





## BERT Extractive Summarizer
Summarize the text using BERT extractive summarizer. This is used to find important sentences and useful sentences from the complete text.<br>
https://pypi.org/project/bert-extractive-summarizer/ <br>
여기 읽으면 코드 이해가 쉬워용 파라미터 설명도 나와있슴니당

In [6]:
## for error solve
# pip install sacremoses

In [13]:
from summarizer import Summarizer

''' https://yurimkoo.github.io/python/2019/09/14/connect-db-with-python.html 참고 
이 부분을 db에서 읽어오기 .. 이런식으로 읽어오면 될 듯 ..?
pip install PyMySQL 설치후

import pymysql

juso_db = pymysql.connect(
    user='root', 
    passwd='{설정한 비밀번호}', 
    host='127.0.0.1', 
    db='juso-db', 
    charset='utf8'
)
cursor = juso_db.cursor(pymysql.cursors.DictCursor)
sql = "SELECT * FROM `busan-jibun`;"
cursor.execute(sql)
result = cursor.fetchall() 
'''
f = open("egypt.txt","r")
full_text = f.read()

model = Summarizer()
result = model(full_text, min_length=60, max_length = 500 , num_sentences = 5)
# result = model(full_text, min_length=60, max_length = 500 , ratio = 0.4)

summarized_text = ''.join(result)
print (summarized_text)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The Nile River fed Egyptian civilization for hundreds of years. This soil was fertile, which means it was good for growing crops. As in many ancient societies, much of the knowledge of Egypt came about as priests studied the world to find ways to please the gods. The Old Kingdom started about 2575 B.C., when the Egyptian empire was gaining strength. This way, the pharaohs hoped to protect their bodies and treasures from robbers.


## Keyword Extraction
Get important keywords from the text and filter those keywords that are present in the summarized text.<br>
https://stackoverflow.com/questions/71838099/typeerror-candidate-selection-got-an-unexpected-keyword-argument-stoplist<br>
참고해서 코드 수정함!(error 나서)<br>
https://boudinfl.github.io/pke/build/html/unsupervised.html<br>
여기 설명이용

In [8]:
import pprint
import itertools
import re
import pke
import string
from nltk.corpus import stopwords

def get_nouns_multipartite(text):
    out=[]

    extractor = pke.unsupervised.MultipartiteRank()
    # extractor.load_document(input=text)

    #    not contain punctuation marks or stopwords as candidates.
    pos = {'PROPN'}
    #pos = {'VERB', 'ADJ', 'NOUN'}
    stoplist = list(string.punctuation)
    stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
    stoplist += stopwords.words('english')

    extractor.load_document(input=text, stoplist=stoplist)
    # extractor.candidate_selection(pos=pos, stoplist=stoplist)
    extractor.candidate_selection(pos=pos)

    # 4. build the Multipartite graph and rank candidates using random walk,
    #    alpha controls the weight adjustment mechanism, see TopicRank for
    #    threshold/method parameters.
    extractor.candidate_weighting(alpha=1.1,
                                  threshold=0.75,
                                  method='average')
    keyphrases = extractor.get_n_best(n=20)

    for key in keyphrases:
        out.append(key[0])

    return out

keywords = get_nouns_multipartite(full_text) 
print (keywords)
filtered_keys=[]
for keyword in keywords:
    if keyword.lower() in summarized_text.lower():
        filtered_keys.append(keyword)
        
print (filtered_keys)

['nile river', 'egyptians', 'egypt', 'nile', 'euphrates', 'tigris', 'old kingdom', 'crown', 'red land', 'longest river', 'narmer united upper', 'africa', 'mediterranean sea', 'hyksos', 'black land', 'new kingdom', 'ethiopia', 'middle kingdom', 'legend', 'lower egypt']
['nile river', 'egyptians', 'egypt', 'nile', 'old kingdom', 'crown', 'red land', 'narmer united upper', 'africa', 'mediterranean sea', 'new kingdom', 'middle kingdom', 'legend', 'lower egypt']


## Sentence Mapping
For each keyword get the sentences from the summarized text containing that keyword. <br>
https://www.nltk.org/api/nltk.tokenize.html<br>
https://flashtext.readthedocs.io/en/latest/keyword_processor.html<br>

In [9]:
## for error solve
# !pip install flashtext

In [10]:
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor

def tokenize_sentences(text):
    sentences = [sent_tokenize(text)]
    sentences = [y for x in sentences for y in x]
    # Remove any short sentences less than 20 letters.
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences

def get_sentences_for_keyword(keywords, sentences):
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in keywords:
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)

    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=True)
        keyword_sentences[key] = values
    return keyword_sentences

sentences = tokenize_sentences(summarized_text)
keyword_sentence_mapping = get_sentences_for_keyword(filtered_keys, sentences)
        
print (keyword_sentence_mapping)

{'nile river': ['The Nile River fed Egyptian civilization for hundreds of years.'], 'egyptians': ['Egyptians cut the stems into strips, pressed them, and dried them into sheets that could be rolled into scrolls.', 'The Nile provided so well for Egyptians that sometimes they had surpluses, or more goods than they needed.', 'For example, some ancient Egyptians learned to be scribes, people whose job was to write and keep records.', 'Egyptians believed that if a tomb was robbed, the person buried there could not have a happy afterlife.', 'This prosperity made life easier and provided greater opportunities for many Egyptians.', 'Egyptians created a government that divided the empire into 42 provinces.', 'Early Egyptians created a hieroglyphic system with about 700 characters.', 'Eventually, Egyptians equipped their reed boats with sails and oars.', 'Poorer Egyptians simply went to the roof to cool off after sunset.', 'Almost all Egyptians married when they were in their early teens.', 'Egy

## Generate MCQ
Get distractors (wrong answer choices) from Wordnet/Conceptnet and generate MCQ Questions.

In [11]:
# for error solve
# pip install -U wn==0.0.22

In [12]:
import requests
import json
import re
import random
from pywsd.similarity import max_similarity
from pywsd.lesk import adapted_lesk
from pywsd.lesk import simple_lesk
from pywsd.lesk import cosine_lesk
from nltk.corpus import wordnet as wn

# Distractors from Wordnet
def get_distractors_wordnet(syn,word):
    distractors=[]
    word= word.lower()
    orig_word = word
    if len(word.split())>0:
        word = word.replace(" ","_")
    hypernym = syn.hypernyms()
    if len(hypernym) == 0: 
        return distractors
    for item in hypernym[0].hyponyms():
        name = item.lemmas()[0].name()
        #print ("name ",name, " word",orig_word)
        if name == orig_word:
            continue
        name = name.replace("_"," ")
        name = " ".join(w.capitalize() for w in name.split())
        if name is not None and name not in distractors:
            distractors.append(name)
    return distractors

def get_wordsense(sent,word):
    word= word.lower()
    
    if len(word.split())>0:
        word = word.replace(" ","_")
    
    
    synsets = wn.synsets(word,'n')
    if synsets:
        wup = max_similarity(sent, word, 'wup', pos='n')
        adapted_lesk_output =  adapted_lesk(sent, word, pos='n')
        lowest_index = min (synsets.index(wup),synsets.index(adapted_lesk_output))
        return synsets[lowest_index]
    else:
        return None

# Distractors from http://conceptnet.io/
def get_distractors_conceptnet(word):
    word = word.lower()
    original_word= word
    if (len(word.split())>0):
        word = word.replace(" ","_")
    distractor_list = [] 
    url = "http://api.conceptnet.io/query?node=/c/en/%s/n&rel=/r/PartOf&start=/c/en/%s&limit=5"%(word,word)
    obj = requests.get(url).json()

    for edge in obj['edges']:
        link = edge['end']['term'] 

        url2 = "http://api.conceptnet.io/query?node=%s&rel=/r/PartOf&end=%s&limit=10"%(link,link)
        obj2 = requests.get(url2).json()
        for edge in obj2['edges']:
            word2 = edge['start']['label']
            if word2 not in distractor_list and original_word.lower() not in word2.lower():
                distractor_list.append(word2)
                   
    return distractor_list

key_distractor_list = {}

for keyword in keyword_sentence_mapping:
    wordsense = get_wordsense(keyword_sentence_mapping[keyword][0],keyword)
    if wordsense:
        distractors = get_distractors_wordnet(wordsense,keyword)
        if len(distractors) ==0:
            distractors = get_distractors_conceptnet(keyword)
        if len(distractors) != 0:
            key_distractor_list[keyword] = distractors
    else:
        
        distractors = get_distractors_conceptnet(keyword)
        if len(distractors) != 0:
            key_distractor_list[keyword] = distractors

index = 1
print ("#############################################################################")
print ("NOTE::::::::  Since the algorithm might have errors along the way, wrong answer choices generated might not be correct for some questions. ")
print ("#############################################################################\n\n")
for each in key_distractor_list:
    sentence = keyword_sentence_mapping[each][0]
    pattern = re.compile(each, re.IGNORECASE)
    output = pattern.sub( " _______ ", sentence)
    print ("%s)"%(index),output)
    choices = [each.capitalize()] + key_distractor_list[each]
    top4choices = choices[:4]
    random.shuffle(top4choices)
    optionchoices = ['a','b','c','d']
    for idx,choice in enumerate(top4choices):
        print ("\t",optionchoices[idx],")"," ",choice)
    print ("\nMore options: ", choices[4:20],"\n\n")
    index = index + 1
    


#############################################################################
NOTE::::::::  Since the algorithm might have errors along the way, wrong answer choices generated might not be correct for some questions. 
#############################################################################


1)  _______  cut the stems into strips, pressed them, and dried them into sheets that could be rolled into scrolls.
	 a )   Egyptians
	 b )   Angolan
	 c )   Algerian
	 d )   Bantu

More options:  ['Basotho', 'Beninese', 'Berber', 'Black African', 'Burundian', 'Cameroonian', 'Carthaginian', 'Chadian', 'Chewa', 'Congolese', 'Djiboutian', 'Egyptian', 'Ethiopian', 'Eurafrican', 'Ewe', 'Fulani'] 


2) Because the pharaoh was thought to be a god, government and religion were not separate in ancient  _______ .
	 a )   Egypt
	 b )   Kuwait
	 c )   Iraq
	 d )   Saudi Arabia

More options:  ['Jordan', 'Israel', 'Fertile Crescent', 'Turkey', 'Iran', 'Lebanon', 'Shari', 'Mauritania', 'Nigeria', 'Somali p